In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install ffmpeg
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy swifter

# !pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110==1.10.0
# !pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110==1.10.0

## !pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110
## !pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
"""from nvidia import dali
dali.__version__"""

'from nvidia import dali\ndali.__version__'

In [ ]:
"""from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.plugin.tf as dali_tf"""
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf_v1
import logging
import os
import glob
import numpy as np
import cupy as cp
import mediapy
from matplotlib import pyplot as plt
import mediapy as media
import pandas as pd
from tqdm import tqdm
import cv2
import swifter
from time import sleep

from tensorflow.keras import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D
from tensorflow.keras.layers import Reshape, AveragePooling2D 
from tensorflow.keras.layers import Dense, Dropout, Input 
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import LSTM, Multiply
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

from tensorflow import optimizers

tf.config.set_soft_device_placement(True)
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false'
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
swifter.__version__

'1.1.2'

In [ ]:
# Paths base
PATH_BASE = '/content/drive/MyDrive/ucf_experiments/'
PATH_CICLE = PATH_BASE + 'ciclo_experimental_1/'

# Path base de armazenamento dos augmentations
PATH_DATA = PATH_CICLE + 'data/'

# Dados de treino augmentados
PATH_AUG_BASE = PATH_DATA + 'augmented_train/'
# Dados de teste redimensionados (visto que não popdem ser augmentados)
PATH_TEST_RESIZED = PATH_DATA + 'resized_test/'

PATH_CROP = PATH_DATA + 'data_crop/'

PATH_CROP_TRAIN = PATH_CROP + 'train_resized.txt'
PATH_CROP_TEST = PATH_CROP + 'test_resized.txt'


In [ ]:
# Com o nome da Base

# pd.read_csv(PATH_TEST_RESIZED+'x_test_resized.csv')
# pd.read_csv(PATH_AUG_BASE + 'x_train_augmented.csv')

In [ ]:
# Create target directory & all intermediate directories if don't exists
def create_dir(dirName):
  try:
      os.makedirs(dirName) 
      print(dirName)   
      print("Directory " , dirName ,  " Created ")
      return dirName
  except FileExistsError:
      print("Directory " , dirName ,  " already exists")  
      return dirName


PATH_CROP_MODEL = create_dir(PATH_CROP + 'model/')
print(PATH_CROP_MODEL)
PATH_CROP_MODEL_CHECKPOINTS = create_dir(PATH_CROP_MODEL + 'checkpoints/')
PATH_TENSORBOARD = create_dir(PATH_CROP_MODEL + 'runs/')
print(PATH_TENSORBOARD)

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/runs/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/runs/


# Pré processamento dos dados

In [ ]:
def filter_small_videos(row):
  cap = cv2.VideoCapture(row['video_path'])
  totalframecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  label = row['label']

  if totalframecount < 32:

    row['small'] = 1
  else:
    row['small'] = 0

  return row

In [ ]:
tqdm.pandas()

## Treino

In [ ]:
videos = pd.read_csv(PATH_CROP + 'train_vFilter_min32frames.txt', names=['video_path', 'label'], sep=';')
videos

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
...,...,...
19335,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
19336,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
19337,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
19338,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


In [ ]:
#videos = pd.read_csv(PATH_CROP + 'train.txt', sep=';', names=['video_path', 'label'])

videos = videos.swifter.apply(filter_small_videos, axis=1)

Pandas Apply:   0%|          | 0/21135 [00:00<?, ?it/s]

In [ ]:
videos.to_csv(PATH_CROP + 'train_vQtFrames.txt', header=None, index=False, sep=';')

In [ ]:
samples = videos.sample(10)
samples

,video_path,label,small
4028,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
11752,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
13797,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
19440,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
5952,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
3242,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
1046,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
2880,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
3625,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
14555,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0


In [ ]:
videos = pd.read_csv(PATH_CROP + 'train_vQtFrames.txt', names=['path', 'label', 'is_small'], sep=';', header=None)

In [ ]:
(videos.is_small == 1).sum()

2872

In [ ]:
videos = {}

for key, video in samples.iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

In [ ]:
media.show_videos(videos)

4028_0 This browser does not support the video tag.,11752_1 This browser does not support the video tag.,13797_1 This browser does not support the video tag.,5952_1 This browser does not support the video tag.,3242_1 This browser does not support the video tag.,1046_0 This browser does not support the video tag.,2880_1 This browser does not support the video tag.,3625_0 This browser does not support the video tag.,14555_1 This browser does not support the video tag.


In [ ]:
#videos['14100_1'].shape

KeyError: ignored

In [ ]:
#test_video = videos['14100_1'][:31]

In [ ]:
"""len_frames = len(videos['14100_1'])
interval = int(np.ceil(len_frames/64)) 

print(len_frames, interval)"""

In [ ]:
"""sampled_video = []
for i in range(0,len_frames,interval):
  print(i, len_frames, interval)
  sampled_video.append(videos['14100_1'][i])"""

In [ ]:
"""num_pad = 64 - len(sampled_video)
if num_pad>0:
    padding = [videos['14100_1'][i] for i in range(-num_pad,0)]
    sampled_video += padding """

In [ ]:
#len(sampled_video)

In [ ]:
#TEste new pipeline

## Validando Treino

In [ ]:
df_videos = pd.read_csv(PATH_CROP + 'train_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0


In [ ]:
df_videos.label.value_counts()

1    10675
0    10460
Name: label, dtype: int64

In [ ]:
df_videos.shape

(21135, 3)

In [ ]:
videos = {}
for key, video in df_videos.loc[df_videos.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

29
30
26
16
24
11
24
30
18
19


In [ ]:
videos = {}
for key, video in df_videos.loc[df_videos.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

125
150
33
246
73
98
150
150
118
56


In [ ]:
df_videos_filtered = df_videos.loc[df_videos.frames32 == 0]
df_videos_filtered.shape

(19340, 3)

In [ ]:
df_videos_filtered.label.value_counts()

0    9815
1    9525
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

18084_0 This browser does not support the video tag.,20194_1 This browser does not support the video tag.,3286_1 This browser does not support the video tag.,8976_1 This browser does not support the video tag.,10371_1 This browser does not support the video tag.,15310_1 This browser does not support the video tag.,17658_0 This browser does not support the video tag.


In [ ]:
df_videos_filtered[['video_path', 'label']].to_csv(PATH_CROP + 'train_vFilter_min32frames.txt', header=None, index=False, sep=';')

## Teste

In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])
videos_teste

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
...,...,...
1052,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1053,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1054,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1055,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


In [ ]:
videos_teste.shape

(1057, 2)

In [ ]:
videos_teste.label.value_counts()

0    550
1    507
Name: label, dtype: int64

In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])

videos_teste = videos_teste.swifter.apply(filter_small_videos, axis=1)

Pandas Apply:   0%|          | 0/1057 [00:00<?, ?it/s]

In [ ]:
videos_teste.to_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, index=False, sep=';')

## Validando Teste

In [ ]:
df_videos_teste = pd.read_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos_teste.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,0
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,0
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,1
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,1


In [ ]:
df_videos_teste.head().video_path.tolist()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/Fighting051_x264-e2c36a76ea_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/Assault014_x264-ebcf10cc63_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/Fighting002_x264-8104c4ee2c_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/Fighting035_x264-b8048b8f3a_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/Assault026_x264-8465a9c86f_1_resized_crop.mp4']

In [ ]:
#cap = media.read_video('/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Fighting051_x264-e2c36a76ea_1_resized_crop.mp4')

In [ ]:
#tf.image.resize(cap, [224,224]).numpy().shape

In [ ]:
df_videos_teste.label.value_counts()

0    550
1    507
Name: label, dtype: int64

In [ ]:
df_videos_teste.shape

(1057, 3)

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

20
26
31
19
19
19
29
9
31
16


In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

55
180
150
78
150
206
69
203
505
150


In [ ]:
df_videos_teste_filtered = df_videos_teste.loc[df_videos_teste.frames32 == 0]
df_videos_teste_filtered.shape

(965, 3)

In [ ]:
df_videos_teste_filtered.label.value_counts()

0    511
1    454
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

1035_0 This browser does not support the video tag.,395_0 This browser does not support the video tag.,46_1 This browser does not support the video tag.,315_0 This browser does not support the video tag.,976_0 This browser does not support the video tag.,324_0 This browser does not support the video tag.,116_1 This browser does not support the video tag.


In [ ]:
df_videos_teste_filtered[['video_path', 'label']].to_csv(PATH_CROP + 'test_vFilter_min32frames.txt', header=None, index=False, sep=';')

## Uniform Sample

In [ ]:
PATH_CROP_UNIFORM_SAMPLE_TRAIN = create_dir(PATH_CROP + 'uniform_samples/train/')
print(PATH_CROP_UNIFORM_SAMPLE_TRAIN)

PATH_CROP_UNIFORM_SAMPLE_TEST = create_dir(PATH_CROP + 'uniform_samples/test/')
print(PATH_CROP_UNIFORM_SAMPLE_TEST)

PATH_CROP_UNIFORM_SAMPLE_CHECK_TRAIN = create_dir(PATH_CROP + 'uniform_samples/checkpoints_train/')
print(PATH_CROP_UNIFORM_SAMPLE_CHECK_TRAIN)

PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST = create_dir(PATH_CROP + 'uniform_samples/checkpoints_test/')
print(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST)

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/train/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/train/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/test/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/test/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/checkpoints_train/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/checkpoints_train/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/checkpoints_test/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/checkpoints_test/


### Treino

In [ ]:
videos_us_train = pd.read_csv(PATH_CROP + 'train_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling(row):
    path_save=PATH_CROP_UNIFORM_SAMPLE_TRAIN
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_train.shape)
display(videos_us_train.head(2))

(19340, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_train, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CROP_UNIFORM_SAMPLE_CHECK_TRAIN+'{}_train_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [4:13:56<00:00, 1523.68s/it]


In [ ]:
#videos_us_train = videos_us_train.progress_apply(lambda x : uniform_sampling(x), axis=1)

In [ ]:
train_parts = glob.glob(PATH_CROP_UNIFORM_SAMPLE_CHECK_TRAIN + '*_checkpoint.csv')
files_train = pd.concat([ pd.read_csv(path) for path in train_parts ])
files_train.shape

(19340, 4)

In [ ]:
files_train.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...
1,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...


In [ ]:
files_train.to_csv(PATH_CROP + 'train_uniform_sample.txt', header=None, index=False, sep=';')

### Teste

In [ ]:
videos_us_test = pd.read_csv(PATH_CROP + 'test_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling_test(row):
    path_save=PATH_CROP_UNIFORM_SAMPLE_TEST
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_test.shape)
display(videos_us_test.head(2))

(965, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_test, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling_test, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST+'{}_test_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [14:46<00:00, 88.62s/it]


In [ ]:
test_parts = glob.glob(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST + '*_checkpoint.csv')
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(965, 4)

In [ ]:
files_test.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...
1,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...


In [ ]:
files_test.to_csv(PATH_CROP + 'test_uniform_sample.txt', header=None, index=False, sep=';')

In [ ]:
from imutils.video import FileVideoStream

In [ ]:
video = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/test/SJRnWfHAx_E_1-a77047fb0d_0_resized_crop.mp4'
fvs = FileVideoStream(video)

In [ ]:
fvs.read()

In [ ]:
# time.sleep(1.0)

# # start the FPS timer
# fps = FPS().start()

# # loop over frames from the video file stream
# while fvs.running():
# 	# grab the frame from the threaded video file stream, resize
# 	# it, and convert it to grayscale (while still retaining 3
# 	# channels)
# 	frame = fvs.read()

# 	# Relocated filtering into producer thread with transform=filterFrame
# 	#  Python 2.7: FPS 92.11 -> 131.36
# 	#  Python 3.7: FPS 41.44 -> 50.11
# 	#frame = filterFrame(frame)

# 	# display the size of the queue on the frame
# 	cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
# 		(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)	

# 	# show the frame and update the FPS counter
# 	cv2.imshow("Frame", frame)

# 	cv2.waitKey(1)
# 	if fvs.Q.qsize() < 2:  # If we are low on frames, give time to producer
# 		time.sleep(0.001)  # Ensures producer runs now, so 2 is sufficient
# 	fps.update()

# Treino

In [ ]:
import numpy as np
import os
from time import time
import cv2

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.utils import Sequence
from keras.utils import np_utils

class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, input_file, batch_size=1, shuffle=True, data_augmentation=True, uniform_sample=False):
        # Initialize the params
        self.batch_size = batch_size
        self.input_file = input_file
        self.classes = 2
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.uniform_sample = uniform_sample
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
        
    def search_data_v2(self):
        X_path = []
        Y_dict = {}
        """one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n')
              path = line[:-2]

              label = int(line[-1:])
              X_path.append(path)
              Y_dict[path] = one_hots[label]"""

        # # list all kinds of sub-folders
        # self.dirs = sorted(os.listdir(self.directory))
        # one_hots = np_utils.to_categorical(range(len(self.dirs)))
        # for i,folder in enumerate(self.dirs):
        #     folder_path = os.path.join(self.directory,folder)
        #     for file in os.listdir(folder_path):
        #         file_path = os.path.join(folder_path,file)
        #         # append the each file path, and keep its label  
        #         X_path.append(file_path)
        #         Y_dict[file_path] = one_hots[i]

        df_videos = pd.read_csv(self.input_file, header=None, sep=';', names=['video_path', 'label', 'frames32'])
        df_videos_filtered = df_videos.loc[df_videos.frames32 == 0]

        X_path = df_videos_filtered[['video_path', 'label']].video_path.to_list()
        Y_dict = pd.get_dummies(df_videos_filtered[['video_path', 'label']], columns=['label']).set_index('video_path').T.to_dict(orient="list")

        return X_path, Y_dict

    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n').split(';')
              path = line[0]

              label = int(line[1])
              X_path.append(path)
              Y_dict[path] = one_hots[label]

        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = self.classes
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(range(self.classes)):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
      
    def normalize(self, data):
        mean = np.mean(data)
        std = np.std(data)
        return (data-mean) / std
    
    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(m=video, axis=2)
        return video    
    
    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval 
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and 
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])     
        # calculate numer of padded frames and fix it 
        num_pad = target_frames - len(sampled_video)
        if num_pad>0:
            padding = [video[i] for i in range(-num_pad,0)]
            sampled_video += padding     
        # get sampled video

        #sampled_video = tf.image.resize(sampled_video, [224,224])
        return np.array(sampled_video, dtype=np.float32)
    
    def dynamic_crop(self, video):
        # extract layer of optical flow from video
        opt_flows = video[...,3]
        # sum of optical flow magnitude of individual frame
        magnitude = np.sum(opt_flows, axis=0)
        # filter slight noise by threshold 
        thresh = np.mean(magnitude)
        magnitude[magnitude<thresh] = 0
        # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
        x_pdf = np.sum(magnitude, axis=1) + 0.001
        y_pdf = np.sum(magnitude, axis=0) + 0.001
        # normalize PDF of x and y so that the sum of probs = 1
        x_pdf /= np.sum(x_pdf)
        y_pdf /= np.sum(y_pdf)
        # randomly choose some candidates for x and y 
        x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
        y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
        # get the mean of x and y coordinates for better robustness
        x = int(np.mean(x_points))
        y = int(np.mean(y_points))
        # avoid to beyond boundaries of array
        x = max(56,min(x,167))
        y = max(56,min(y,167))
        # get cropped video 
        return video[:,x-56:x+56,y-56:y+56,:]  
    
    def color_jitter(self,video):
        # range of s-component: 0-1
        # range of v component: 0-255
        s_jitter = np.random.uniform(-0.2,0.2)
        v_jitter = np.random.uniform(-30,30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
            s = hsv[...,1] + s_jitter
            v = hsv[...,2] + v_jitter
            s[s<0] = 0
            s[s>1] = 1
            v[v<0] = 0
            v[v>255] = 255
            hsv[...,1] = s
            hsv[...,2] = v
            video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        return video
        
    def load_data(self, path):
        data = media.read_video(path)[...,:3]
        data = np.float32(data)
        # sampling 64 frames uniformly from the entire video

        if self.uniform_sample:
          data = self.uniform_sampling(video=data, target_frames=64)
        else:
          data = np.array(data, dtype=np.float32)

        # whether to utilize the data augmentation
        if  self.data_aug:
            data = self.color_jitter(data)
            data = self.random_flip(data, prob=0.5)
        # normalize

        data = self.normalize(data)
        return data

In [ ]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation, LeakyReLU, Add, Multiply
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
def create_model(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(64,224,224,3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)

  #####################################################
  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)

  #####################################################
  x = Flatten()(x)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(2, activation='softmax')(x)
  model = Model(inputs=inputs, outputs=pred)

  adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [ ]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    # Every 10 epochs, the learning rate is reduced to 1/10 of the original
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.5)
    return K.get_value(model.optimizer.lr)


In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
# parallel_model = multi_gpu_model(model, gpus=2)

In [ ]:
# dataframe_train = pd.read_csv(PATH_CROP + 'train_uniform_sample.txt', sep=';', names=['label', 'uniform_paths'])
# dataframe_test = pd.read_csv(PATH_CROP + 'test_uniform_sample.txt', sep=';', names=['label', 'uniform_paths'])

In [ ]:
"""max_batch_size = 5

dataframe_train = pd.read_csv(PATH_CROP + 'train_uniform_sample.txt', sep=';', names=['old_path', 'label', 'uniform_paths'])
dataframe_test = pd.read_csv(PATH_CROP + 'test_uniform_sample.txt', sep=';', names=['old_path', 'label', 'uniform_paths'])

train_dataset_file_list = dataframe_train.uniform_paths.tolist()
train_dataset_label_list = dataframe_train.label.tolist()

test_dataset_file_list = dataframe_test.uniform_paths.tolist()
test_dataset_label_list = dataframe_test.label.tolist()

checkpoint_path = PATH_CROP_MODEL_CHECKPOINTS + "model.{epoch:02d}.h5"
tb_path = PATH_TENSORBOARD + 'tensorboard'

BATCH_SIZE=2
EPOCHS = 200
ITERATIONS_PER_EPOCH = int(len(train_dataset_file_list)/BATCH_SIZE) #cada iteração possui 8 vídeos
ITER=100

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

shapes = ((BATCH_SIZE, SequenceLength, IMSIZE[0], IMSIZE[1], 3), (BATCH_SIZE, N_CLASSES))
dtypes = (tf.float32, tf.float32)

@pipeline_def
def video_pipe(file_list, label_list):
    videos, label = fn.readers.video(device="gpu", filenames = file_list, labels=label_list, sequence_length=SequenceLength, lazy_init=True,
                                    shard_id=0, num_shards=1, file_list_include_preceding_frame=True, random_shuffle=True, 
                                     normalized=True, image_type=types.RGB,
                                    dtype=types.FLOAT, pad_sequences=True)

    label = fn.one_hot(label, num_classes=N_CLASSES)

    return videos, label.gpu()"""

In [ ]:
# import cv2 
# from google.colab.patches import cv2_imshow

# import numpy as np
# video = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/uniform_samples/test/SJRnWfHAx_E_1-a77047fb0d_0_resized_crop.mp4'

# %time
# cap = cv2.VideoCapture(video)

# boolean, frame = cap.read()

# print(frame)

In [ ]:
# %time
# data = media.read_video(video)[...,:3]
# data = np.float32(data)
# data

In [ ]:
# if last_checkpoint < 10:
#   model = load_model(PATH_CROP_MODEL_CHECKPOINTS + "model.0{}.h5".format(str(last_checkpoint)))
# else:
#   model = load_model(PATH_CROP_MODEL_CHECKPOINTS + "model.{}.h5".format(str(last_checkpoint)))

In [ ]:


with tf.device('/gpu:0'):
  num_epochs  = 30
  num_workers = 16
  batch_size  = 8

  # pipe_train = video_pipe(file_list=train_dataset_file_list, label_list=train_dataset_label_list, batch_size=BATCH_SIZE, num_threads=4, device_id=0)
  # pipe_test = video_pipe(file_list=test_dataset_file_list, label_list=test_dataset_label_list, batch_size=BATCH_SIZE, num_threads=4, device_id=0)

  # train_generator = dali_tf.DALIDataset(
  #     pipeline=pipe_train,
  #     batch_size=BATCH_SIZE,
  #     output_shapes=shapes,
  #     output_dtypes=dtypes,
  #     device_id=0
  # )

  # val_generator = dali_tf.DALIDataset(
  #     pipeline=pipe_test,
  #     batch_size=BATCH_SIZE,
  #     output_shapes=shapes,
  #     output_dtypes=dtypes,
  #     device_id=0
  # )

  train_generator = DataGenerator(input_file=PATH_CROP + 'train_uniform_sample.txt', 
                                batch_size=batch_size, 
                                data_augmentation=False,
                                uniform_sample=False)

  val_generator = DataGenerator(input_file=PATH_CROP + 'test_uniform_sample.txt',
                                batch_size=batch_size, 
                                data_augmentation=False,
                                uniform_sample=False)
  
  model = create_model()

  reduce_lr = LearningRateScheduler(scheduler)
  tb_callback = tf.keras.callbacks.TensorBoard(PATH_TENSORBOARD, update_freq=1)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=PATH_CROP_MODEL_CHECKPOINTS + "model.{epoch:02d}.h5",
                                                  save_weights_only=True,
                                                  verbose=1)
  
  callbacks_list = [reduce_lr, tb_callback, cp_callback]
  
  hist = model.fit_generator(
    generator=train_generator, 
    validation_data=val_generator,
    callbacks=callbacks_list,
    verbose=1, 
    epochs=num_epochs,
    workers=num_workers,
    max_queue_size=8,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Found 19340 files belonging to 2 classes.
         0 :  0
         1 :  1
Found 965 files belonging to 2 classes.
         0 :  0
         1 :  1


KeyboardInterrupt: ignored

In [ ]:
22:46


## Fazer resize
## Fazer uniform sampling + resize
## ...
## Regerar extração de bouding box

salvar tensorboard (graficos)
salvar checkpoinys do modelo
colocar as métricas de f1, recall, precision, auc

In [ ]:
dfg

In [ ]:
#dataset = 'RWF2000-opt'



In [ ]:
"""train_generator = DataGenerator(input_file=PATH_CROP + 'train_vFilter_min32frames.txt', 
                              batch_size=batch_size, 
                              data_augmentation=False)

for batch, labels in tqdm(train_generator, total=len(train_generator)):
  if batch[0].shape[2] != 224:
    print('small width')"""

Found 19340 files belonging to 2 classes.
         0 :  0
         1 :  1


100%|██████████| 2418/2418 [5:01:34<00:00,  7.48s/it]


In [ ]:
"""val_generator = DataGenerator(input_file=PATH_CROP + 'test_vFilter_min32frames.txt',
                              batch_size=batch_size, 
                              data_augmentation=False)

for batch, labels in tqdm(val_generator, total=len(val_generator)):
  if batch[0].shape[2] != 224:
    print(batch[0].shape)"""

Found 965 files belonging to 2 classes.
         0 :  0
         1 :  1


  1%|          | 1/121 [00:04<08:53,  4.45s/it]

(64, 112, 112, 3)


  2%|▏         | 2/121 [00:08<07:52,  3.97s/it]

(64, 112, 112, 3)


...

(64, 112, 112, 3)


 99%|█████████▉| 120/121 [08:27<00:04,  4.05s/it]

(64, 112, 112, 3)


100%|██████████| 121/121 [08:30<00:00,  4.22s/it]

(64, 112, 112, 3)


In [ ]:
FIM

In [ ]:
max_batch_size = 5
dataset_file_list = os.path.join(PATH_CROP, 'train_resized.txt')
checkpoint_path = PATH_CROP_MODEL_CHECKPOINTS + "model.{epoch:02d}.h5"
tb_path = PATH_CROP_MODEL + 'tensorboard'

EPOCHS = 200
ITERATIONS_PER_EPOCH = 512 #cada iteração possui 8 vídeos
BATCH_SIZE=2
ITER=100

N_CLASSES = 2
IMSIZE = (112, 112)
SequenceLength = 64

shapes = ((BATCH_SIZE, SequenceLength, IMSIZE[0], IMSIZE[1], 3), (BATCH_SIZE, N_CLASSES))
dtypes = (tf.float32, tf.float32)

@pipeline_def
def video_pipe(file_list):
    videos, label = fn.readers.video(device="gpu", file_list = file_list, sequence_length=SequenceLength, lazy_init=True,
                                    shard_id=0, num_shards=1, file_list_include_preceding_frame=True, random_shuffle=True, 
                                     normalized=True, image_type=types.RGB,
                                    dtype=types.FLOAT, pad_sequences=True)

    label = fn.one_hot(label, num_classes=N_CLASSES)

    return videos, label.gpu()

- Modelos manuais
- Modelo do RWF2000
- Classes Removidas: Road Accidents, Arson, Explosion
- Extraido videos com probabilidade >= 0.99
- Mantido segmentos de vídeos com menos de 70 frames 
- Learning rate dinamico
- Videos redimensionados para 224,224
- Data Augmentation: Brilho, Contrasto, Flip, Ajuste de HUE

In [ ]:
def scheduler(epoch):
    with tf.device('/gpu:0'):
      # Every 10 epochs, the learning rate is reduced to 1/10 of the original
      if epoch % 10 == 0 and epoch != 0: #A qt de épocas é divisível por 10? Então, A cada 10 épocas, reduza o learnrate em 1/10 do tamanho
          lr = K.get_value(model.optimizer.lr)
          K.set_value(model.optimizer.lr, lr * 0.5)

      return K.get_value(model.optimizer.lr)

In [ ]:
## Ref. https://www.programcreek.com/python/example/120287/keras.layers.Conv3D
## Ref. https://arxiv.org/pdf/1911.05913v3.pdf
def create_model() -> tf.keras.Model:
  with tf.device('/gpu:0'):
    add_model = Sequential()
    in_shape = (SequenceLength, IMSIZE[0], IMSIZE[1], 3)

    add_model.add(Input(shape=in_shape))
    metrics=[
             tf.keras.metrics.Precision(),
             tf.keras.metrics.CategoricalAccuracy(),
             tf.keras.metrics.Recall(),
             tf.keras.metrics.AUC()
            ]

    # add_model.add(ConvLSTM2D(filters=16, kernel_size=(3,3),
    #     padding='same', return_sequences=True))
    # add_model.add(BatchNormalization())

    add_model.add(Conv3D(
        filters=16, kernel_size=(1,3,3),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=16, kernel_size=(3,1,1),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(1, 2, 2)))

    add_model.add(Conv3D(
        filters=16, kernel_size=(1,3,3),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=16, kernel_size=(3,1,1),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(1, 2, 2)))

    add_model.add(Conv3D(
        filters=32, kernel_size=(1,3,3),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=32, kernel_size=(3,1,1),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(1, 2, 2)))

    
    # REMOVENDO ALGUMAS CAMADA POIS AS DIMENSÕES DO VÍDEO JÁ SÃO PEQUENAS
    add_model.add(Conv3D(
        filters=32, kernel_size=(1,3,3),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=32, kernel_size=(3,1,1),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(1, 2, 2)))

    add_model.add(MaxPooling3D(pool_size=(8, 1, 1)))

    add_model.add(Conv3D(
        filters=64, kernel_size=(1,3,3),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=64, kernel_size=(3,1,1),
        strides=(1,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    add_model.add(Conv3D(
        filters=64, kernel_size=(1,3,3),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(Conv3D(
        filters=64, kernel_size=(3,1,1),
        padding="same", kernel_initializer='he_normal', activation="relu"))
    add_model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    # add_model.add(Conv3D(
    #     filters=128, kernel_size=(1,3,3),
    #     strides=(1,1,1),
    #     padding="same", kernel_initializer='he_normal', activation="relu"))
    # add_model.add(Conv3D(
    #     filters=128, kernel_size=(3,1,1),
    #     strides=(1,1,1),
    #     padding="same", kernel_initializer='he_normal', activation="relu"))
    # add_model.add(MaxPooling3D(pool_size=(2, 3, 3)))

    add_model.add(Flatten())
    add_model.add(Dense(128, activation='relu'))
    add_model.add(Dropout(.2))
    add_model.add(Dense(32, activation='relu'))
    add_model.add(Dense(N_CLASSES, activation='softmax'))

    model = add_model

    sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=sgd,
                  metrics=metrics)
    
    return model

In [ ]:
pipe = video_pipe(file_list=dataset_file_list, batch_size=BATCH_SIZE, num_threads=4, device_id=0)

In [ ]:
crime_ds = dali_tf.DALIDataset(
        pipeline=pipe,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)

In [ ]:
crime_ds

In [ ]:
with tf.device('/gpu:0'):
  # Create a callback that saves the model's weights
  reduce_lr_callback = LearningRateScheduler(scheduler)
  tb_callback = tf.keras.callbacks.TensorBoard(tb_path, update_freq=1)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
  early_stop =  tf.keras.callbacks.EarlyStopping(patience=2)
 
  pipe = video_pipe(file_list=dataset_file_list, batch_size=BATCH_SIZE, num_threads=4)
  
  crime_ds = dali_tf.DALIDataset(
        pipeline=pipe,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)
  
  model = create_model()
  print(model.summary())
  hist = model.fit(
        crime_ds,
        epochs=EPOCHS,
        workers=16,
        max_queue_size=8, 
        steps_per_epoch=ITERATIONS_PER_EPOCH,
        callbacks=[
                    reduce_lr_callback,
                    tb_callback, 
                    cp_callback,
                    early_stop
                   ])

In [ ]:
def show_videos(videos, labels):
  with tf.device('/gpu:0'):
    labeled_videos = {}

    for index, label in enumerate(labels):
      video = videos[index]

      class_int = int(label.argmax())
      labeled_videos[str(index) + '_' + classes[class_int]] = video

    mediapy.show_videos(labeled_videos)


In [ ]:
@pipeline_def
def test_video_pipe(file_list):
    videos, label = fn.readers.video(device="gpu", file_list = file_list, sequence_length=SequenceLength, lazy_init=True,
                                    shard_id=0, num_shards=1, normalized=True, image_type=types.RGB,
                                    dtype=types.FLOAT, pad_sequences=True)

    label = fn.one_hot(label, num_classes=N_CLASSES)

    return videos, label.gpu()

In [ ]:
TEST_BATCH_SIZE=4

def dataset_fn(ds_path):
    with tf.device("/gpu:0"):
      pipe = test_video_pipe(file_list=ds_path, batch_size=BATCH_SIZE, num_threads=4)  
      crime_ds = dali_tf.DALIDataset(
            pipeline=pipe,
            batch_size=BATCH_SIZE,
            output_shapes=shapes,
            output_dtypes=dtypes,
            device_id=0)
      
      return crime_ds

In [ ]:
test_dataset_path=output_test_path
test_dataset = dataset_fn(test_dataset_path)

In [ ]:
def show_videos_labels(videos, labels):
  videos_pair = {}

  for idx, pack in enumerate(zip(videos, labels)):
    video, label = pack
    key = str(idx) + '_' + str(CLASSES[np.argmax(label)])

    videos_pair[key] = np.array(video)

  return videos_pair 

In [ ]:
validation_df = pd.DataFrame()

with tf.device("/gpu:0"):
  model = create_model()
  model.load_weights('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/model/checkpoints/model.196.h5')
  
  for idx, payload in enumerate(test_dataset):


    items, labels = payload
    predictions = model.predict_on_batch(items)
    K.clear_session()

    batch = {
        'true':np.argmax(predictions, axis=1),
        'predictions':np.argmax(labels, axis=1)
    }

    _temp = pd.DataFrame(batch)

    validation_df = validation_df.append(_temp, ignore_index=True)
    if idx % 100 == 0:
      print(idx)
      
    if idx == 1000:
      break

validation_df.to_csv('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/rwf_wy_dynamic_crop_conv3d_tf.csv')

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_pred = validation_df['predictions'], y_true=validation_df['true'])

In [ ]:
print(report)

In [ ]:
with tf.device('/gpu:0'):
  model = create_model()
  model.load_weights('/content/drive/MyDrive/hr_crimes/checkpoints/model.40.h5')

  model.evaluate(test_dataset, batch_size=BATCH_SIZE, steps=ITERATIONS_PER_EPOCH)